In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Analytic continuation

:::{note}

Improvements to analytic continuation in AmpForm are currently being developed in {doc}`compwa-report:003/index` and {doc}`compwa-report:004/index`.

:::

Analytic continuation allows one to handle resonances just below threshold ($m_0 < m_a + m_b$  in Eq. {eq}`relativistic_breit_wigner_with_ff`). In practice, this entails using a specific function for $\rho$ in Eq. {eq}`EnergyDependentWidth`.

## Definitions

Three usual choices for $\rho$ are the following:

In [ ]:
import timeit
import warnings
from textwrap import dedent
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from IPython.display import Markdown, Math
from matplotlib_inline.backend_inline import set_matplotlib_formats
from sympy.printing.latex import LatexPrinter

from ampform.dynamics.phasespace import BreakupMomentum
from ampform.io import aslatex
from ampform.kinematics.phasespace import Kallen
from ampform.sympy import unevaluated


def display_doit(exprs: list[sp.Expr], deep: bool = True) -> Math:
    return Math(aslatex({x: x.doit(deep=deep) for x in exprs}))


set_matplotlib_formats("svg")
warnings.filterwarnings("ignore")

### 1) Break-up momentum

The {func}`~sympy.functions.elementary.miscellaneous.sqrt` or {class}`.ComplexSqrt` of {class}`.BreakupMomentumSquared`:

In [ ]:
from ampform.dynamics import BreakupMomentumSquared

s, m_a, m_b = sp.symbols("s, m_a, m_b", nonnegative=True)
q_squared = BreakupMomentumSquared(s, m_a, m_b)
Math(aslatex({q_squared: q_squared.evaluate()}))

### 2) 'Normal' phase space factor

The 'normal' {class}`.PhaseSpaceFactor` (the denominator makes the difference to {eq}`EnergyDependentWidth`!):

In [ ]:
from ampform.dynamics import PhaseSpaceFactor

rho = PhaseSpaceFactor(s, m_a, m_b)
Math(aslatex({rho: rho.evaluate()}))

### 3) 'Complex' phase space factor

A {class}`.PhaseSpaceFactorComplex` that uses {class}`.ComplexSqrt`:

In [ ]:
from ampform.dynamics import PhaseSpaceFactorComplex

rho_c = PhaseSpaceFactorComplex(s, m_a, m_b)
Math(aslatex({rho_c: rho_c.evaluate()}))

### 4) 'Analytic continuation' of the phase space factor

The following 'case-by-case' **analytic continuation** for decay products with an _equal_ mass, {class}`.EqualMassPhaseSpaceFactor`:

In [ ]:
from ampform.dynamics import EqualMassPhaseSpaceFactor

rho_ac = EqualMassPhaseSpaceFactor(s, m_a, m_b)
Math(aslatex({rho_ac: rho_ac.evaluate()}))

with

In [ ]:
from ampform.dynamics import PhaseSpaceFactorAbs

rho_hat = PhaseSpaceFactorAbs(s, m_a, m_b)
Math(aslatex({rho_hat: rho_hat.evaluate()}))

(Mind the absolute value.)

### 5) Chew-Mandelstam for $S$-waves

A {class}`.PhaseSpaceFactorSWave` that uses {func}`.chew_mandelstam_s_wave`:

In [ ]:
from ampform.dynamics import PhaseSpaceFactorSWave

rho_cm = PhaseSpaceFactorSWave(s, m_a, m_b)
Math(aslatex({rho_cm: rho_cm.evaluate()}))

## Cut structure

When continued into the complex plane, the breakup momentum and phase space factor exhibit distinct cut structures, depending on the definition of the square root in the numerator. The separated square root definition,

In [ ]:
m1, m2 = sp.symbols("m1 m2", nonnegative=True)
display_doit([
    BreakupMomentum(s, m1, m2),
    PhaseSpaceFactor(s, m1, m2),
])

leads to a cleaner cut structure than a definition with a "single" square root,

In [ ]:
@unevaluated
class BreakupMomentumSingleSqrt(sp.Expr):
    s: Any
    m1: Any
    m2: Any

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt((s - (m1 - m2) ** 2) * (s - (m1 + m2) ** 2)) / (2 * sp.sqrt(s))

    def _latex_repr_(self, printer: LatexPrinter, *args) -> str:
        s = self.args[0]
        s_latex = printer._print(s)
        return Rf"q\left({s_latex}\right)"


@unevaluated
class PhaseSpaceFactorSingleSqrt(sp.Expr):
    s: Any
    m1: Any
    m2: Any

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt((s - (m1 - m2) ** 2) * (s - (m1 + m2) ** 2)) / s

    def _latex_repr_(self, printer: LatexPrinter, *args) -> str:
        s = self.args[0]
        s_latex = printer._print(s)
        return Rf"\rho\left({s_latex}\right)"


display_doit([
    BreakupMomentumSingleSqrt(s, m1, m2),
    PhaseSpaceFactorSingleSqrt(s, m1, m2),
])

or with a Källén function,

In [ ]:
@unevaluated
class BreakupMomentumKallen(sp.Expr):
    s: Any
    m1: Any
    m2: Any

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2) / (4 * s))

    def _latex_repr_(self, printer: LatexPrinter, *args) -> str:
        s = self.args[0]
        s_latex = printer._print(s)
        return Rf"q\left({s_latex}\right)"


@unevaluated
class PhaseSpaceFactorKallen(sp.Expr):
    s: Any
    m1: Any
    m2: Any

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / s

    def _latex_repr_(self, printer: LatexPrinter, *args) -> str:
        s = self.args[0]
        s_latex = printer._print(s)
        return Rf"\rho\left({s_latex}\right)"


display_doit([
    BreakupMomentumKallen(s, m1, m2),
    PhaseSpaceFactorKallen(s, m1, m2),
])

In [ ]:
x_min, x_max = -0.1, +1.5
y_max = 0.8
z_max = 0.5
x = np.linspace(x_min, x_max, num=500)
X, Y = np.meshgrid(x, np.linspace(-y_max, +y_max, num=300))
S = X + 1j * Y
ϵi = 1e-7j

parameters = {m1: 0.2, m2: 0.6}
thr_neg = (parameters[m1] - parameters[m2]) ** 2
thr_pos = (parameters[m1] + parameters[m2]) ** 2

fig, axes = plt.subplots(
    dpi=200,
    figsize=(12, 5),
    ncols=3,
    nrows=2,
    sharex=True,
    sharey=True,
)
fig.subplots_adjust(bottom=0, hspace=0.02, left=0, right=1, top=1, wspace=0.02)
fig.patch.set_facecolor("none")
for ax in axes.flatten():
    ax.patch.set_facecolor("none")
    ax.spines["bottom"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.hlines(0, thr_neg, thr_pos, color="black", lw=1.5, zorder=10)
    ax.scatter([thr_neg, thr_pos], [0, 0], color="black", s=25, zorder=10)
for ax in axes[-1]:
    ax.set_xlabel(R"$\mathrm{Re}\,s$", labelpad=-12)
    ax.set_xticks([0])
for ax in axes[:, 0]:
    ax.set_ylabel(R"$\mathrm{Im}\,s$")
    ax.set_ylim(-y_max, +y_max)
    ax.set_yticks([0])

style = dict(
    cmap="coolwarm",
    rasterized=True,
    vmin=-z_max,
    vmax=+z_max,
    zorder=-10,
)
for ax, expr_class, title in [
    (axes[0, 0], BreakupMomentum, '$q(s)$, "split" square root'),
    (axes[0, 1], BreakupMomentumSingleSqrt, "$q(s)$, single square root"),
    (axes[0, 2], BreakupMomentumKallen, "$q(s)$ with Källén function"),
    (axes[1, 0], PhaseSpaceFactor, R'$\rho(s)$, "split" square root'),
    (axes[1, 1], PhaseSpaceFactorSingleSqrt, R"$\rho(s)$, single square root"),
    (axes[1, 2], PhaseSpaceFactorKallen, R"$\rho(s)$ with Källén function"),
]:
    expr = expr_class(s, m1, m2)
    func = sp.lambdify(s, expr.doit().subs(parameters))
    mesh = ax.pcolormesh(X, Y, func(S).imag, **style)
    ax.plot(x, func(x + ϵi).real, c="darkblue")
    ax.plot(x, func(x + ϵi).imag, c="darkgreen")
    ax.set_title(title, y=0.9)
cbar = fig.colorbar(mesh, ax=axes, pad=0.01)
cbar.ax.set_ylabel("real", labelpad=0, rotation=270)
cbar.ax.set_yticks([-z_max, +z_max])
cbar.ax.set_yticklabels(["$-$", "$+$"])
ax = axes[0, 0]
kwargs = dict(transform=ax.transAxes, ha="right", va="top")
ax.text(0.99, 0.48, "real", c="darkblue", **kwargs)
ax.text(0.99, 0.97, "imag", c="darkgreen", **kwargs)
plt.show()

## Numerical precision and performance

The numerical precision of functions decreases as the number of operations (nodes) in the function tree increases. As such, AmpForm's standard {class}`.BreakupMomentum` has a slightly lower numerical precision and is slower, even if its cut structure is cleaner.

The implementation of the breakup momentum and the phasespace factor that uses the Källén function and decreases numerical precision by a factor&nbsp;$10$.

In [ ]:
def render_floating_point_differences(exprs: dict[str, sp.Expr]) -> None:
    m1_val, m2_val = parameters.values()
    n_eval = 1_000
    fig, ax = plt.subplots(figsize=(7, 3))
    ax.spines["bottom"].set_position("zero")
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    y_max = 0
    src = dedent(f"""
    | Name | Operations | Mean abs. difference | Standard deviation | {n_eval} evaluations (ms) |
    |:-----|-----------:|---------------------:|-------------------:|------:|
    """)
    for name, expr in exprs.items():
        n_ops = sp.count_ops(expr.doit())
        func = sp.lambdify(args, expr.doit())
        exact = [expr.doit().subs(parameters).subs(s, x).n() for x in s_array]
        numerical = func(s_array, m1_val, m2_val)
        diff = numerical - np.array(exact, dtype=float)
        diff_abs = np.abs(diff)
        mean = sp.sympify(diff_abs.mean()).n(3)
        std = sp.sympify(diff_abs.std()).n(3)
        run = lambda: func(test_array, m1_val, m2_val)  # noqa:B023,E731
        ms = 1e3 * np.array(timeit.repeat(run, number=n_eval, repeat=10))
        time = Rf"${ms.mean():.1f} \pm {ms.std():.1f}$"
        src += f"| {name} | {n_ops:,d} | ${sp.latex(mean)}$ | ${sp.latex(std)}$ | {time} |\n"
        y_max = max(y_max, diff_abs[3:].max())
        ax.fill_between(s_array, diff_abs, alpha=0.7, label=name, step="mid")
    ax.set_ylim(0, y_max)
    ax.legend()
    ax.set_xlabel("$s$")
    ax.set_ylabel("Abs. diff. with algebraic value")
    fig.tight_layout()
    plt.show()
    display(Markdown(src))


args = s, m1, m2
s_array = np.linspace(thr_pos + 1e-3, x_max, num=50)
test_array = np.linspace(0, 10, num=1000)

In [ ]:
render_floating_point_differences({
    "Källén function": PhaseSpaceFactorKallen(*args),
    "Split square root": PhaseSpaceFactor(*args),
    "Single square root": PhaseSpaceFactorSingleSqrt(*args),
})

In [ ]:
render_floating_point_differences({
    "Källén function": BreakupMomentumKallen(*args),
    "Split square root": BreakupMomentum(*args),
    "Single square root": BreakupMomentumSingleSqrt(*args),
})

## Interactive visualization

```{autolink-skip}
```

In [ ]:
%matplotlib widget

In [ ]:
import symplot
from ampform.sympy.math import ComplexSqrt

m = sp.Symbol("m", nonnegative=True)
rho_c = PhaseSpaceFactorComplex(m**2, m_a, m_b)
rho_cm = PhaseSpaceFactorSWave(m**2, m_a, m_b)
rho_ac = EqualMassPhaseSpaceFactor(m**2, m_a, m_b)
np_rho_c, sliders = symplot.prepare_sliders(plot_symbol=m, expression=rho_c.doit())
np_rho_ac = sp.lambdify((m, m_a, m_b), rho_ac.doit())
np_rho_cm = sp.lambdify((m, m_a, m_b), rho_cm.doit())
np_breakup_momentum = sp.lambdify(
    (m, m_a, m_b),
    2 * ComplexSqrt(q_squared.subs(s, m**2).doit()),
)

{{ run_interactive }}

In [ ]:
plot_domain = np.linspace(0, 3, 500)
sliders.set_ranges(
    m_a=(0, 2, 200),
    m_b=(0, 2, 200),
)
sliders.set_values(
    m_a=0.3,
    m_b=0.75,
)

In [ ]:
import mpl_interactions.ipyplot as iplt

import symplot

fig, axes = plt.subplots(
    ncols=2,
    nrows=2,
    figsize=[8, 5],
    sharex=True,
    sharey=True,
)
fig.subplots_adjust(bottom=0.08, hspace=0.1, left=0.01, right=0.99, top=1, wspace=0.05)
fig.canvas.footer_visible = False
fig.canvas.header_visible = False
fig.canvas.toolbar_visible = False
fig.patch.set_facecolor("none")
for ax in axes.flatten():
    ax.patch.set_facecolor("none")
    ax.spines["bottom"].set_position("zero")
    ax.spines["left"].set_position("zero")
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)

(ax_q, ax_rho), (ax_rho_ac, ax_rho_cm) = axes
for ax in axes[-1]:
    ax.set_xlabel("$m$")
for ax in axes.flatten():
    ax.set_yticks([])
    ax.set_yticks([])


def func_imag(func, *args, **kwargs):
    return lambda *args, **kwargs: func(*args, **kwargs).imag


def func_real(func, *args, **kwargs):
    return lambda *args, **kwargs: func(*args, **kwargs).real


ylim = (-0.1, 1.4)
q_math = ComplexSqrt(sp.Symbol("q^2")) / (8 * sp.pi)
ax_q.set_title(f"${sp.latex(q_math)}$", y=0.85)
controls = iplt.plot(
    plot_domain,
    func_real(np_breakup_momentum),
    label="real",
    **sliders,
    ylim=ylim,
    ax=ax_q,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_breakup_momentum),
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=ax_q,
    alpha=0.7,
)

ax_rho.set_title(f"${sp.latex(rho_c)}$", y=0.85)
iplt.plot(
    plot_domain,
    func_real(np_rho_c),
    label="real",
    controls=controls,
    ylim=ylim,
    ax=ax_rho,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_c),
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=ax_rho,
    alpha=0.7,
)

ax_rho_ac.set_title(R"equal mass $\rho^\mathrm{eq}(m^2)$", y=0.85)
iplt.plot(
    plot_domain,
    func_real(np_rho_ac),
    label="real",
    controls=controls,
    ylim=ylim,
    ax=ax_rho_ac,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_ac),
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=ax_rho_ac,
    alpha=0.7,
)

ax_rho_cm.set_title(R"Chew-Mandelstam $\rho^\mathrm{CM}(m^2)$", y=0.85)
iplt.plot(
    plot_domain,
    func_real(np_rho_cm),
    label="real",
    controls=controls,
    ylim=ylim,
    ax=ax_rho_cm,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_cm),
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=ax_rho_cm,
    alpha=0.7,
)

plt.legend(loc="lower right")
plt.show()

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import SVG, display

    output_file = "analytic-continuation.svg"
    plt.savefig(output_file)
    display(SVG(output_file))